<a href="https://colab.research.google.com/github/eduardo2531/lab_1/blob/main/Semestral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instale un montón de bibliotecas
!pip3 install neat-python
!pip3 install gym-retro
!pip3 install numpy
!pip3 install cloudpickle
!pip3 install opencv-python
!pip3 install matplotlib
!pip3 install scikit-image
!pip3 install virtualenv
!pip3 install pygame


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Ahora necesitarás importar todas las bibliotecas.

In [ ]:
import numpy as np
import retro
import neat
import pickle
import cv2
import pygame 
# video
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-white')
 
# Procesamento de imagenes
from skimage.transform import resize
from skimage.color import rgb2gray
from IPython import display

Creamos las carpetas del openai retro, clonamos el repositorio de python usando la opcion de fork.
creamos la virtual y la activamos 


In [ ]:
mkdir baselines 

In [ ]:
python3 -m venv baselines

In [ ]:
cd baselines 

In [ ]:
source bin/activate

In [ ]:
pip freeze

In [ ]:
! git clone --recursive https://github.com/openai/baselines.git gym-retro

In [ ]:
! git clone --recursive https://github.com/dsapandora/sonic_controller_data_capture.git

In [ ]:
virtualenv -p python3 env

In [ ]:
source env/bin/activate

In [ ]:
pip3 install gym-retro

In [ ]:
import retro

def main():
    env = retro.make(game='Airstriker-Genesis')
    obs = env.reset()
    while True:
        obs, rew, done, info = env.step(env.action_space.sample())
        env.render()
        if done:
            obs = env.reset()
    env.close()


if __name__ == "__main__":
    main()

¿Qué juegos ya se han integrado? Tenga en cuenta que esto mostrará todos los entornos definidos, incluso aquellos para los que faltan ROM.
importracion de la roms

In [ ]:
python3 -m retro.import /ROMs/directory/